## Data Generation - Analytical Method

In [1]:
import math
import numpy
import pandas as pd

In [2]:
Pmax = 100000

V1 = 400
V2 = 400
f = 50000
Coss = 5*10**(-9)
max_phi = 0.8 * math.pi
rows = []

L = 4*V1*V2*math.sin(0.8*math.pi)/(Pmax*(math.pi)**(3)*f)

for power in range(10000,100000,10000):
    for dutycycle1 in range(0,98,1):
        d1 = dutycycle1/100 * math.pi
        
        for dutycycle2 in range(0,98,1):
            d2 = dutycycle2/100 * math.pi
            
            sinmin = (power*(math.pi)**(3)*f*L)/(4*V1*V2*math.cos(d1/2)*math.cos(d2/2))
            
            if (sinmin >= -1 and sinmin <= 1):
                phi2 = math.asin(sinmin)
                                
                v1_total_p1 = 0
                v2_total_p1 = 0
                
                i_12total_p1 = 0
                i_s = []
                omega = 2*math.pi*f
                for x1 in range (0,200,10):
                    x1 = x1/100 * math.pi
                    for nn in range (1,50,2):
                    
                        L12 = L
                    
                        d1r = 4/math.pi * math.cos(nn*d1/2)
                        d2r = 4/math.pi * math.cos(nn*d2/2) * math.cos(nn*phi2)
                        d2i = 4/math.pi * math.cos(nn*d2/2) * math.sin(nn*phi2)
                        
                        i_12_p1 = -V2/ (L12 * omega) * (d1r * math.cos(nn*x1*omega)/(nn**2) 
                                                        - d2r * math.cos(nn*x1*omega)/(nn**2) 
                                                        + d2i * math.sin(nn*x1 * omega)/(nn**2))
                    
                        i_12total_p1 = i_12total_p1 + i_12_p1
                    
                    i_s.append([abs(i_12total_p1)])
                i_max = max(i_s)
                i_s.clear()

                rows.append([phi2, d1, d2, power, i_max[0]])   
            else:
                continue
                
df = pd.DataFrame(rows,columns=["phi","d1","d2" ,"P","I"])
# df.to_csv('lookup-table_test1.csv', index=False)
df.head(5)

,phi,d1,d2,P,I
0,0.058812,0.0,0.000000,10000,379.629089
1,0.058820,0.0,0.031416,10000,379.271925
2,0.058841,0.0,0.062832,10000,378.144862
3,0.058878,0.0,0.094248,10000,376.101166
4,0.058929,0.0,0.125664,10000,372.955244


In [3]:
## Test purpose
test_power = 30000
test_value = df[df['P']==test_power]['I'].min()
print('Minimum current for Power={} Watts is {} Amperes'.format(test_power, 
                                                                 test_value))

Minimum current for Power=30000 Watts is 1069.282711735033 Amperes


### Grouping Power and Filtering data with Minimum Current Stress ($I_{L_{min}}$)

In [4]:
# Grouping entire dataset with similar power values and filtering out for minimum current stress
df2 = df.groupby(['P'])['I'].min()    
df_final = df.loc[df['I'].isin(df2.values)]
df_final.head(5)

,phi,d1,d2,P,I
2933,0.074106,0.942478,0.942478,10000,351.715352
8878,0.122180,0.000000,0.534071,20000,715.101337
16820,0.187481,0.000000,0.659734,30000,1069.282712
23958,0.252564,0.000000,0.691150,40000,1428.153616
30381,0.333407,0.000000,0.911062,50000,1788.485699


### Saving filtered Data to CSV file

In [5]:
df_final.to_csv('TPSC_DAB_dataset.csv', index=False)

### Adding more test dataset for model evaluation

In [6]:
# df_test1 = pd.read_csv('lookup-table_test1.csv')
# df_test2 = pd.read_csv('lookup-table_test2.csv')
# df_test = pd.concat([df_test1, df_test2], ignore_index=True, sort=False)
# df_test.to_csv('lookup_table_test.csv')